In [1]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests, json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries Imported!")

Libraries Imported!


## Question 1

## Scrape data from HTML tables into a DataFrame

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")
table = soup.find('table')

toronto_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
for row in soup.find('table').find_all('td'):
    if row.span.text=='Not assigned':
        pass
    else:
        postalCode = row.p.text[:3]
        borough = (row.span.text).split('(')[0]
        neighborhood = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        toronto_data = toronto_data.append({"PostalCode":postalCode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)    

toronto_data.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [3]:
toronto_data = toronto_data.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
toronto_data.head()        

PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [4]:
print("Shape: ", toronto_data.shape)

Shape:  (103, 3)


## Question 2: Add coordinates data

In [5]:
coordinates = pd.read_csv("./Geospatial_Coordinates.csv")
coordinates.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [6]:
df = toronto_data.join(coordinates.set_index('Postal Code'), on='PostalCode')
df.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## Question 3: Explore and cluster the neighborhoods in Toronto

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare Credentials and Version

In [9]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'

#### Explore Neighborhoods in Toronto

In [10]:
# Set the limit of venues
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(1559, 7)


Neighborhood  Neighborhood Latitude  \
0                          Malvern, Rouge              43.806686   
1  Rouge Hill, Port Union, Highland Creek              43.784535   
2       Guildwood, Morningside, West Hill              43.763573   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0              -79.194353                Wendy’s       43.807448   
1              -79.160497  Royal Canadian Legion       43.782533   
2              -79.188711         RBC Royal Bank       43.766790   
3              -79.188711      G & G Electronics       43.765309   
4              -79.188711             Sail Sushi       43.765951   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.191151                  Bank  
3       -79.191537     Electronics Store  
4       -79.191275            Restaurant

### Analyze Each Neighborhood

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0            0               0        0                   0             0   
1            0               0        0                   0             0   
2            0               0        0                   0             0   
3            0               0        0                   0             0   
4            0               0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0             0  ...      0              0                              0   
1             0  ...      0              0                              0   
2             0  ...      0              0                              0   
3             0  ...      0              0                              0   
4             0  ...      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0                 0                      0                0         0   
1                 0                      0                0         0   
2                 0                      0                0         0   
3                 0                      0                0         0   
4                 0                      0                0         0   

   Wine Shop  Wings Joint  Women's Store  
0          0            0              0  
1          0            0              0  
2          0            0              0  
3          0            0              0  
4          0            0              0  

[5 rows x 245 columns]

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                          Agincourt          0.0   
1                             Alderwood, Long Branch          0.0   
2    Bathurst Manor, Wilson Heights, Downsview North          0.0   
3                                    Bayview Village          0.0   
4                  Bedford Park, Lawrence Manor East          0.0   
..                                               ...          ...   
96                           Willowdale, Newtonbrook          0.0   
97                                            Woburn          0.0   
98                                  Woodbine Heights          0.0   
99                                   York Mills West          0.0   
100                         York Mills, Silver Hills          0.0   

     Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0               0.0      0.0                 0.0           0.0   
1               0.0      0.0                 0.0           0.0   
2               0.0      0.0                 0.0           0.0   
3               0.0      0.0                 0.0           0.0   
4               0.0      0.0                 0.0           0.0   
..              ...      ...                 ...           ...   
96              0.0      0.0                 0.0           0.0   
97              0.0      0.0                 0.0           0.0   
98              0.0      0.0                 0.0           0.0   
99              0.0      0.0                 0.0           0.0   
100             0.0      0.0                 0.0           0.0   

     Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0.0              0.0               0.0             0.000000   
1               0.0              0.0               0.0             0.000000   
2               0.0              0.0               0.0             0.000000   
3               0.0              0.0               0.0             0.000000   
4               0.0              0.0               0.0             0.041667   
..              ...              ...               ...                  ...   
96              0.0              0.0               0.0             0.000000   
97              0.0              0.0               0.0             0.000000   
98              0.0              0.0               0.0             0.000000   
99              0.0              0.0               0.0             0.000000   
100             0.0              0.0               0.0             0.000000   

     ...  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0    ...    0.0            0.0                            0.0   
1    ...    0.0            0.0                            0.0   
2    ...    0.0            0.0                            0.0   
3    ...    0.0            0.0                            0.0   
4    ...    0.0            0.0                            0.0   
..   ...    ...            ...                            ...   
96   ...    0.0            0.0                            0.0   
97   ...    0.0            0.0                            0.0   
98   ...    0.0            0.0                            0.0   
99   ...    0.0            0.0                            0.0   
100  ...    0.0            0.0                            0.0   

     Video Game Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0                 0.0                    0.0              0.0       0.0   
1                 0.0                    0.0              0.0       0.0   
2                 0.0                    0.0              0.0       0.0   
3                 0.0                    0.0              0.0       0.0   
4                 0.0                    0.0              0.0       0.0   
..                ...                    ...              ...       ...   
96                0.0                    0.0              0.0       0.0   
97                0.0                    0.0              0.0       0.0   
98           

### Let's confirm the new size

In [15]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1               Skating Rink  0.25
2  Latin American Restaurant  0.25
3             Breakfast Spot  0.25
4        Monument / Landmark  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1             Gym  0.17
2     Coffee Shop  0.17
3  Sandwich Place  0.17
4             Pub  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.11
1     Coffee Shop  0.11
2   Grocery Store  0.06
3       Gift Shop  0.06
4  Ice Cream Shop  0.06


----Bayview Village----
                        venue  freq
0         Japanese Restaurant  0.25
1                        Café  0.25
2                        Bank  0.25
3          Chinese Restaurant  0.25
4  Modern European Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.08
1               Restaurant  0.08
2       

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood      1st Most Common Venue  \
0                                        Agincourt  Latin American Restaurant   
1                           Alderwood, Long Branch                Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North                Coffee Shop   
3                                  Bayview Village         Chinese Restaurant   
4                Bedford Park, Lawrence Manor East           Sushi Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Skating Rink                Lounge        Breakfast Spot   
1           Coffee Shop                   Pub                   Gym   
2                  Bank             Gift Shop   Fried Chicken Joint   
3                  Café                  Bank   Japanese Restaurant   
4        Sandwich Place           Coffee Shop            Restaurant   

  5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
0          Dessert Shop     Electronics Store  Eastern European Restaurant   
1        Sandwich Place        Discount Store                 Dessert Shop   
2        Sandwich Place           Bridal Shop                        Diner   
3               Dog Run                 Diner               Discount Store   
4       Thai Restaurant          Liquor Store                    Juice Bar   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0             Drugstore            Donut Shop       Doner Restaurant  
1    Dim Sum Restaurant                 Diner                Dog Run  
2            Restaurant        Ice Cream Shop       Sushi Restaurant  
3   Distribution Center         Women's Store           Dessert Shop  
4    Italian Restaurant     Indian Restaurant   Fast Food Restaurant

### Cluster Neighborhoods

In [73]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [74]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.dropna()
toronto_merged['Cluster Label'] = toronto_merged['Cluster Label'].astype(int)

toronto_merged.head() # check the last columns!

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Label 1st Most Common Venue  2nd Most Common Venue  \
0 -79.194353              1  Fast Food Restaurant          Women's Store   
1 -79.160497              1                   Bar          Women's Store   
2 -79.188711              1        Medical Center    Rental Car Location   
3 -79.216917              1           Coffee Shop  Korean BBQ Restaurant   
4 -79.239476              1                Bakery                   Bank   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0      Department Store           Escape Room     Electronics Store   
1          Dessert Shop  Ethiopian Restaurant           Escape Room   
2            Restaurant    Mexican Restaurant                  Bank   
3    Mexican Restaurant         Women's Store      Department Store   
4    Athletics & Sports      Hakka Restaurant       Thai Restaurant   

         6th Most Common Venue        7th Most Common Venue  \
0  Eastern European Restaurant                    Drugstore   
1            Electronics Store  Eastern European Restaurant   
2                 Intersection            Electronics Store   
3            Electronics Store  Eastern European Restaurant   
4         Caribbean Restaurant                  Gas Station   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Donut Shop      Doner Restaurant                Dog Run  
1             Drugstore            Donut Shop       Doner Restaurant  
2        Breakfast Spot            Donut Shop    Distribution Center  
3             Drugstore            Donut Shop       Doner Restaurant  
4   Distribution Center    Dim Sum Restaurant                  Diner

Finally, let's visualize the resulting clusters

In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster 1

In [85]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Label       1st Most Common Venue  \
14             Scarborough              0                        Park   
21              North York              0                        Park   
23              North York              0                        Park   
25              North York              0                        Park   
30              North York              0            Business Service   
31              North York              0                        Park   
40   East YorkEast Toronto              0                        Park   
44         Central Toronto              0                        Park   
50        Downtown Toronto              0                        Park   
64         Central Toronto              0                        Park   
72              North York              0           Convenience Store   
74                    York              0                        Park   
79              North York              0  Construction & Landscaping   
90               Etobicoke              0                        Park   
100              Etobicoke              0                        Park   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14    Arts & Crafts Store          Intersection            Playground   
21          Women's Store         Deli / Bodega     Electronics Store   
23      Convenience Store         Women's Store         Deli / Bodega   
25   Fast Food Restaurant     Food & Drink Shop         Women's Store   
30                   Park               Airport   Distribution Center   
31          Grocery Store         Shopping Mall                  Bank   
40      Convenience Store         Women's Store         Deli / Bodega   
44            Swim School              Bus Line      Department Store   
50             Playground                 Trail          Dance Studio   
64                  Trail      Sushi Restaurant         Jewelry Store   
72                   Park            Hookah Bar      Asian Restaurant   
74          Women's Store                   Bar      Department Store   
79       Basketball Court                  Park                Bakery   
90                   Pool                 River          Dance Studio   
100           Pizza Place        Sandwich Place              Bus Line   

           5th Most Common Venue        6th Most Common Venue  \
14           Distribution Center                 Dessert Shop   
21   Eastern European Restaurant                    Drugstore   
23             Electronics Store  Eastern European Restaurant   
25                 Deli / Bodega  Eastern European Restaurant   
30              Department Store                 Dessert Shop   
31              Department Store            Electronics Store   
40             Electronics Store  Eastern European Restaurant   
44             Electronics Store  Eastern European Restaurant   
50   Eastern European Restaurant                    Drugstore   
64                  Dessert Shop           Dim Sum Restaurant   
72                       Dog Run           Dim Sum Restaurant   
74                   Escape Room            Electronics Store   
79                         Diner               Discount Store   
90   Eastern European Restaurant                    Drugstore   
100                 Dance Studio                    Drugstore   

           7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
14            Dim Sum Restaurant                 Diner        Discount Store   
21                    Donut Shop      Doner Restaurant               Dog Run   
23                     Drugstore            Donut Shop      Doner Restaurant   
25                     Drugstore            Donut Shop      Doner Restaurant   
30            Dim Sum Restaurant                 Diner        Discount Store   
31   Eastern European Restaurant             Drugstore            Donut Shop   
40                     Drugstore            Donut Shop      Doner Restaurant   


Cluster 2

In [87]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Label 1st Most Common Venue  \
0           Scarborough              1  Fast Food Restaurant   
1           Scarborough              1                   Bar   
2           Scarborough              1        Medical Center   
3           Scarborough              1           Coffee Shop   
4           Scarborough              1                Bakery   
..                  ...            ...                   ...   
95            Etobicoke              1                  Café   
97           North York              1        Baseball Field   
99            Etobicoke              1            Playground   
101           Etobicoke              1         Grocery Store   
102  EtobicokeNorthwest              1   Rental Car Location   

     2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Women's Store      Department Store           Escape Room   
1            Women's Store          Dessert Shop  Ethiopian Restaurant   
2      Rental Car Location            Restaurant    Mexican Restaurant   
3    Korean BBQ Restaurant    Mexican Restaurant         Women's Store   
4                     Bank    Athletics & Sports      Hakka Restaurant   
..                     ...                   ...                   ...   
95                Pharmacy            Beer Store          Liquor Store   
97             Fabric Shop         Women's Store          Dessert Shop   
99      Chinese Restaurant          Intersection        Sandwich Place   
101            Coffee Shop        Sandwich Place            Beer Store   
102              Drugstore         Garden Center          Dance Studio   

    5th Most Common Venue        6th Most Common Venue  \
0       Electronics Store  Eastern European Restaurant   
1             Escape Room            Electronics Store   
2                    Bank                 Intersection   
3        Department Store            Electronics Store   
4         Thai Restaurant         Caribbean Restaurant   
..                    ...                          ...   
95      Convenience Store                  Pizza Place   
97   Ethiopian Restaurant                  Escape Room   
99         Discount Store    Middle Eastern Restaurant   
101              Pharmacy                  Pizza Place   
102     Electronics Store  Eastern European Restaurant   

           7th Most Common Venue        8th Most Common Venue  \
0                      Drugstore                   Donut Shop   
1    Eastern European Restaurant                    Drugstore   
2              Electronics Store               Breakfast Spot   
3    Eastern European Restaurant                    Drugstore   
4                    Gas Station          Distribution Center   
..                           ...                          ...   
95                Shopping Plaza                    Pet Store   
97             Electronics Store  Eastern European Restaurant   
99                   Coffee Shop                      Dog Run   
101         Fast Food Restaurant                 Dessert Shop   
102                   Donut Shop             Doner Restaurant   

    9th Most Common Venue 10th Most Common Venue  
0        Doner Restaurant                Dog Run  
1              Donut Shop       Doner Restaurant  
2              Donut Shop    Distribution Center  
3              Donut Shop       Doner Restaurant  
4      Dim Sum Restaurant                  Diner  
..                    ...                    ...  
95                   Park            Coffee Shop  
97              Drugstore             Donut Shop  
99    Distribution Center       Department Store  
101    Dim Sum Restaurant                  Diner  
102               Dog Run    Distribution Center  

[84 rows x 12 columns]

Cluster 3

In [88]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Label 1st Most Common Venue 2nd Most Common Venue  \
96  North York              2                   Gym         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
96         Deli / Bodega           Escape Room     Electronics Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
96  Eastern European Restaurant             Drugstore            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
96      Doner Restaurant                Dog Run

Cluster 4

In [90]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Label 1st Most Common Venue 2nd Most Common Venue  \
98    York              3     Convenience Store         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
98      Department Store           Escape Room     Electronics Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
98  Eastern European Restaurant             Drugstore            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
98      Doner Restaurant                Dog Run